In [ ]:
#region imports
from AlgorithmImports import *
#endregion

'''
This strategy is based on
https://quantpedia.com/strategies/asset-class-trend-following/
'''


class MomentumAssetAllocationStrategy(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetCash(100000)

        self.data = {}
        period = 6 #12 * 0.5
        self.SetWarmUp(period, Resolution.Daily)

        self.traded_count = 3
        self.symbols = ["SPY", "EFA", "IEF", "VNQ", "GSG"]
        for symbol in self.symbols:
            self.AddEquity(symbol, Resolution.Minute)
            self.data[symbol] = self.ROC(symbol, period, Resolution.Daily)

        self.recent_month = -1
        self.Plot("Performance", "Portfolio Value", self.Portfolio.TotalPortfolioValue)

    def OnData(self, data):
        if self.IsWarmingUp: return
        if not (self.Time.hour == 9 and self.Time.minute == 31):
            return
        if self.Time.month == self.recent_month:
            return
        self.recent_month = self.Time.month
        selected = {}
        for symbol, roc in self.data.items():
            if symbol in data and data[symbol] and roc.IsReady:
                selected[symbol] = roc.Current.Value
                self.Plot("Momentum Scores", symbol, roc.Current.Value)

        if len(selected) < self.traded_count:
            return

        # sorted_by_momentum = sorted(selected.items(), key=lambda x: x[1], reverse=True)
        # long = [x[0] for x in sorted_by_momentum[:self.traded_count]] if len(sorted_by_momentum) >= self.traded_count else []

        sorted_by_momentum = sorted(selected.items(), key=lambda x: x[1], reverse=True)
        long_candidates = sorted_by_momentum[:self.traded_count]

        # Calculate total momentum for normalization
        total_momentum = sum([x[1] for x in long_candidates])

        # invested = [x.Key.Value for x in self.Portfolio if x.Value.Invested]
        # for symbol in invested:
        #     if symbol not in long:
        #         self.Liquidate(symbol)

        # for symbol in long:
        #     self.SetHoldings(symbol, 1 / len(long))
        #     self.Plot("Asset Allocation", symbol, 100 / len(long))

         # Manage existing positions
        for symbol in self.Portfolio.Keys:
            if symbol not in [x[0] for x in long_candidates]:
                self.Liquidate(symbol)

        # Allocate funds based on momentum
        for symbol, momentum in long_candidates:
            if total_momentum > 0:  # Prevent division by zero
                allocation_percentage = momentum / total_momentum
                self.SetHoldings(symbol, allocation_percentage)
                self.Plot("Asset Allocation", symbol, allocation_percentage * 100)

    def OnEndOfMonth(self):
        self.Plot("Performance", "Portfolio Value", self.Portfolio.TotalPortfolioValue)

